<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguage Natural

## Taller #10: Twitter
### Ilvar Dario Sanabria. Código: 616201058
`Fecha de entrega: Noviembre 14, 2020 (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

###  `[25 pts]` Punto 1: Extraer Tuits
Extraer mínimo 100 tuits de Twitter usando la API

(Si exponen sus claves, les haré maldades 😈)

❌🎃❌ ¡No Halloween!

In [2]:
# Instalamos las librearias
import pandas as pd
import tweepy

In [3]:
# Leer las Claves para acceder al api de twiter

keys = pd.read_csv("c:/tmp/KeysAPI_twiter.txt", header=None)
keys = dict(zip(keys[0],keys[1]))

In [4]:
# Solicitar el acceso a través API

auth = tweepy.OAuthHandler(keys['api_key'], keys['api_secret_key'])
auth.set_access_token(keys['access_token'], keys['access_token_secret'])

api = tweepy.API(auth, wait_on_rate_limit=True)

In [5]:
# Leemos las coord de ciudades desde un archivo

ciudades = pd.read_csv("c:/tmp/co_small.csv")
ciudades.head()

,city,lat,lng,country,iso2,admin,capital,population,population_proper
0,Bogotá,4.649178,-74.062827,Colombia,CO,Bogotá,primary,7772000.0,6333661.0
1,Medellín,6.251840,-75.563591,Colombia,CO,Antioquia,admin,3297000.0,1999979.0
2,Cali,3.437222,-76.522500,Colombia,CO,Valle del Cauca,admin,2254000.0,2178836.0
3,Barranquilla,10.963889,-74.796389,Colombia,CO,Atlántico,admin,1798000.0,1244491.0
4,Bucaramanga,7.125393,-73.119804,Colombia,CO,Santander,admin,1009000.0,571820.0


In [6]:
# Recolectamos los datos relacinados con la palabra #selección#
all_tweets = []

for ciudad in ciudades.iterrows() :
    c = ciudad[1]
    gc = f"{c.lat},{c.lng},70km"
    c_name = c.city
      
    tweets = tweepy.Cursor(api.search,
                        q=['selección -RT'],
                        lang='es',
                        geocode=gc,
                        tweet_mode='extended',
                        include_rts=False).items(20)


    t = []
 
    for tweet in tweets:
        temp = []
        temp.append(tweet.full_text)
        temp.append(tweet.created_at)
        temp.append(c_name)
        temp.append(tweet.user.location)
        temp.append(tweet.id)
        temp.append(f"https://twitter.com/i/web/status/{tweet.id}")
 
        t.append(temp)
    
    all_tweets.extend(t)
    print(c_name)
    

Bogotá
Medellín
Cali
Barranquilla
Bucaramanga
Cartagena
Cúcuta
Pereira
Santa Marta
Ibagué
Pasto
Manizales
Villavicencio
Neiva
Armenia
Valledupar
Montería
Sincelejo
Popayán
Tunja
Ríohacha
Florencia
Quibdó
Arauca
Yopal
Leticia
San Andrés
San José del Guaviare
Mocoa
Puerto Carreño
Mitú
Inírida


In [7]:
#Creamos un dataFrame con los datos recolectados
datos_tw = pd.DataFrame(all_tweets)
datos_tw.columns = ['tweet', 'date', 'city', 'user_location', 'id', 'link']

datos_tw.head()

,tweet,date,city,user_location,id,link
0,#Colombia | El inigualable @santiagoarias13 al...,2020-11-13 21:19:53,Bogotá,Bogotá - Colombia,1327360559579353088,https://twitter.com/i/web/status/1327360559579...
1,Con esos 30 grados se ve más cansada la selecc...,2020-11-13 21:19:30,Bogotá,bogota,1327360465660481537,https://twitter.com/i/web/status/1327360465660...
2,Entonces ahora que diran algunos del tema inte...,2020-11-13 21:19:18,Bogotá,Colombia,1327360412593975297,https://twitter.com/i/web/status/1327360412593...
3,Se supone que la estrategia de jugar en Barran...,2020-11-13 21:19:06,Bogotá,Bogotá,1327360363646607361,https://twitter.com/i/web/status/1327360363646...
4,"Uribe no pesa, no genera, no filtra. Muy poco ...",2020-11-13 21:18:46,Bogotá,"Bogotá, Colombia",1327360278682537987,https://twitter.com/i/web/status/1327360278682...


In [8]:
#Se define la funcion de limpieza de datos
import re
 
pattern1 = '?P<pic>pic.twitter.com/[^\s]+'
pattern2 = '?P<url>https?://[^\s]+'
 
def text_clean(row):    
    text = row['tweet']
    links = [tuple(j for j in i if j)[-1] for i in re.findall(f"({pattern1})|({pattern2})",text)]

    for link in links:
        text = text.replace(link,"")

    hashtags = [interaction for interaction in text.split() if interaction.startswith("#")]

    for hashtag in hashtags:
         text = text.replace(hashtag,"")

    mentions = [interaction for interaction in text.split() if interaction.startswith("@")]

    for mention in mentions:
         text = text.replace(mention,"")

    return text, links, hashtags, mentions

In [9]:
datos_tw[['clean', 'links', 'hashtags', 'mentions']] = datos_tw.apply(text_clean, axis=1, result_type='expand')

datos_tw.head()

,tweet,date,city,user_location,id,link,clean,links,hashtags,mentions
0,#Colombia | El inigualable @santiagoarias13 al...,2020-11-13 21:19:53,Bogotá,Bogotá - Colombia,1327360559579353088,https://twitter.com/i/web/status/1327360559579...,| El inigualable alentó a la Selección. Más ...,"[https://t.co/y6snbl2krJ, https://t.co/1CnwX7R...",[#Colombia],[@santiagoarias13]
1,Con esos 30 grados se ve más cansada la selecc...,2020-11-13 21:19:30,Bogotá,bogota,1327360465660481537,https://twitter.com/i/web/status/1327360465660...,Con esos 30 grados se ve más cansada la selecc...,[],"[#Colombia, #golcaracol, #ColombiaVsUruguay]",[]
2,Entonces ahora que diran algunos del tema inte...,2020-11-13 21:19:18,Bogotá,Colombia,1327360412593975297,https://twitter.com/i/web/status/1327360412593...,Entonces ahora que diran algunos del tema inte...,[],[],[]
3,Se supone que la estrategia de jugar en Barran...,2020-11-13 21:19:06,Bogotá,Bogotá,1327360363646607361,https://twitter.com/i/web/status/1327360363646...,Se supone que la estrategia de jugar en Barran...,[],[#SeleccionColombia],[@FCFSeleccionCol]
4,"Uribe no pesa, no genera, no filtra. Muy poco ...",2020-11-13 21:18:46,Bogotá,"Bogotá, Colombia",1327360278682537987,https://twitter.com/i/web/status/1327360278682...,"Uribe no pesa, no genera, no filtra. Muy poco ...",[],[],[]


###  `[25 pts]` Punto 2: Hacer una visualización sobre esos tuits

❌🎃❌ ¡No Halloween!

In [10]:
## Mostramos los emojis que estan en los twits
import regex

import emoji
 
def get_emojis(text):
    emoji_list = []

    data = regex.findall(r'\X', text)

    for word in data:
        if any(char in emoji.UNICODE_EMOJI for char in word):
             emoji_list.append(word)
 
    return emoji_list
 
datos_tw['emojis'] = datos_tw['clean'].apply(lambda text: get_emojis(text))

In [11]:
datos_tw['emojis']

0      []
1      []
2      []
3      []
4      []
       ..
500    []
501    []
502    []
503    []
504    []
Name: emojis, Length: 505, dtype: object

In [12]:
## Importamos las librerias necesarias para realizar una grafica
import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
 
import matplotlib.pyplot as plt

%matplotlib inline

init_notebook_mode(connected=True)

In [13]:
## Imprimimos una serie de tiempo
datos_tw['fecha'] = datos_tw.date.dt.date

temp = pd.DataFrame(datos_tw.fecha.value_counts()).reset_index()

temp.columns = ['fecha', 'cnt']
temp = temp.sort_values('fecha')

temp

,fecha,cnt
9,2020-11-04,1
8,2020-11-05,1
3,2020-11-06,8
6,2020-11-07,5
5,2020-11-08,7
7,2020-11-09,5
4,2020-11-10,7
2,2020-11-11,16
1,2020-11-12,19
0,2020-11-13,436


In [14]:
trace = go.Scatter(x=temp.fecha.values,
                    y=temp.cnt.values,
                    text=[f"Fecha: {f}<br>Tuits: {c}" for f,c in zip(temp.fecha.values,temp.cnt.values)],
                    hoverinfo='text',
                    mode='lines+markers',
                    name='Días',
                    line={'color':'#1DA1F2'})
 
layout = go.Layout(title='Número de tuits por día') 
fig = go.Figure(data=[trace], layout=layout)

iplot(fig)

In [15]:
### Diagrama de barras de la cantidad de emojis

In [16]:
from collections import Counter
 
temp_emojis = pd.DataFrame(list(zip(Counter(sum(datos_tw.emojis.values,[])).keys(),Counter(sum(datos_tw.emojis.values,[])).values())))

temp_emojis.columns = ['emoji', 'cnt']
temp_emojis.sort_values('cnt', ascending=False, inplace=True)

temp_emojis.head()

,emoji,cnt
0,🇨🇴,46
14,⚽,17
26,🙏,10
2,⚽️,6
1,🇺🇾,6


In [17]:
trace = go.Bar(x=temp_emojis.emoji.values[:15],
                y=temp_emojis.cnt.values[:15])
 
layout = go.Layout(title='Emojis más usados')
 
fig = go.Figure(data=[trace], layout=layout)

iplot(fig)

### Hacemos un Mapa de calor de las palabras 

In [18]:
#Se importan las librerias para quitar las stopwors y hacer TFID 
import re

from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer

stopwords_sp = stopwords.words('spanish')
 
def pre_procesado(texto):
     texto = texto.lower()
     texto = re.sub(r"[\W\d_]+", " ", texto)
     texto = " ".join([palabra for palabra in texto.split() if palabra not in stopwords_sp])
     return texto
 
tfidf_vect = TfidfVectorizer(preprocessor=pre_procesado, ngram_range=(1,2), max_features=50)

tfidf = tfidf_vect.fit_transform(datos_tw.clean.values)

In [19]:
# Se obtienen las distancias entre palabras
from sklearn.metrics.pairwise import cosine_similarity
 
voc = [k for k,v in sorted(tfidf_vect.vocabulary_.items(), key=lambda kv: kv[1])]

temp = pd.DataFrame(tfidf.toarray())
temp = cosine_similarity(temp.T.values)
temp = pd.DataFrame(temp)
temp.columns = voc
temp.index = voc

temp

,así,años,barranquilla,bien,camiseta,colombia,cuadrado,cuadrado selección,debe,díaz,...,ser,si,siempre,tan,uruguay,va,vamos,vamos selección,ver,vez
así,1.000000,0.000000,0.064430,0.000000,0.000000,0.077631,0.049868,0.109550,0.000000,0.000000,...,0.000000,0.089805,0.000000,0.000000,0.039352,0.000000,0.000000,0.000000,0.152552,0.000000
años,0.000000,1.000000,0.000000,0.000000,0.000000,0.077353,0.000000,0.000000,0.000000,0.000000,...,0.037264,0.084990,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
barranquilla,0.064430,0.000000,1.000000,0.000000,0.000000,0.074826,0.000000,0.000000,0.000000,0.000000,...,0.282309,0.000000,0.000000,0.000000,0.067344,0.000000,0.000000,0.000000,0.032694,0.041864
bien,0.000000,0.000000,0.000000,1.000000,0.000000,0.036014,0.070110,0.042552,0.000000,0.000000,...,0.029933,0.085183,0.029649,0.000000,0.079036,0.042543,0.000000,0.000000,0.000000,0.113359
camiseta,0.000000,0.000000,0.000000,0.000000,1.000000,0.061473,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.018955,0.057562,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
colombia,0.077631,0.077353,0.074826,0.036014,0.061473,1.000000,0.002692,0.005913,0.000000,0.124230,...,0.010371,0.051032,0.143485,0.104556,0.263753,0.105168,0.060465,0.050112,0.038053,0.061231
cuadrado,0.049868,0.000000,0.000000,0.070110,0.000000,0.002692,1.000000,0.518739,0.077041,0.000000,...,0.012642,0.039330,0.012522,0.000000,0.008014,0.017968,0.015403,0.020116,0.058220,0.037191
cuadrado selección,0.109550,0.000000,0.000000,0.042552,0.000000,0.005913,0.518739,1.000000,0.000000,0.000000,...,0.000000,0.086400,0.027508,0.000000,0.000000,0.000000,0.000000,0.000000,0.127898,0.000000
debe,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.077041,0.000000,1.000000,0.000000,...,0.170815,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
díaz,0.000000,0.000000,0.000000,0.000000,0.000000,0.124230,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.017809,0.160357,0.000000,0.000000,0.000000,0.086408


#### Se hace la grafica de calor con base en las distancias

In [20]:
import scipy.cluster.hierarchy as sch

import numpy as np
 
pairwise_distances = sch.distance.pdist(temp)
linkage = sch.linkage(pairwise_distances, method='ward')

idx_to_cluster_array = sch.fcluster(linkage, pairwise_distances.max() * 1, criterion='distance')
idx = np.argsort(idx_to_cluster_array)

temp = temp.copy()

temp2 = temp.iloc[idx, :].T.iloc[idx, :]

my_idx = idx_to_cluster_array

In [22]:
trace = go.Heatmap(z=temp2.values.tolist(),
                    x=temp2.index.values,
                    y=temp2.columns.values,
                    colorscale='Blues')
 
layout = go.Layout(title=' Mapa de calor entre palabras - Twits relacionados con la "Selección"', width=800, height=800)
 
fig = go.Figure(data=[trace],layout=layout)

iplot(fig)